Todo: 
- Dataset
- CNN
- CV


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
import pickle

### Load in data

In [5]:
input_path = "C:/Users/Matt/Dropbox/SnowComp/ModisImages.npy"
dataset = np.load(input_path)

path_ids = "C:/Users/Matt/Dropbox/SnowComp/cell_ids.pkl"
with open(path_ids, 'rb') as handle:
    cell_ids = pickle.load( handle)

train_y = pd.read_csv("C:/Users/Matt/Documents/Python Scripts/SnowComp/dat/train_labels.csv")

In [6]:
#labels helpers and processing
def pivot_df(df, id_col, ignore_cols=None):
    if not ignore_cols:
        ignore_cols = []
    date_cols = [x for x in df.columns if x not in [id_col] + ignore_cols]
    dfs = []
    for day in date_cols:
        day_df = df[[id_col, day]].rename({day: 'snowpack'}, axis=1)
        day_df['date'] = day
        dfs.append(day_df)
    return pd.concat(dfs)

def daynum_gen(date_time):
    '''converts date time objects to filename'''
    date_time = datetime.fromisoformat(date_time)
    doy = date_time.timetuple().tm_yday
    year = date_time.year
    return str(year) + '{:03d}'.format(doy)

train_y = pivot_df(train_y, 'cell_id').dropna()
train_y['date']=train_y['date'].map(daynum_gen)

In [7]:
#sort train_y so it has correct order before stripping labels
train_y['idx'] = train_y['cell_id'] +"-"+train_y['date']
sorter = [iden +"-" +date for  iden, date  in cell_ids]
train_y = train_y.set_index('idx')
train_y = train_y.loc[sorter]

In [12]:
dataset.shape
train_y.shape

(91490, 3)

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

np.random.seed(111)
tf.random.set_seed(111)

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential


#define model
model = Sequential()

model.add(Conv2D(30, kernel_size=5, activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, kernel_size=4, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(5, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1, activation='relu'))

print(model.summary())


rmse = tf.keras.metrics.RootMeanSquaredError()
model.compile(loss='mean_squared_error', optimizer="adam", metrics=[rmse])

SyntaxError: invalid syntax (Temp/ipykernel_6680/3871919539.py, line 10)